# Create Schema

In [ ]:
display(dbutils.fs.ls("/"))

path,name,size,modificationTime
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/flights_data/,flights_data/,0,1724836507000
dbfs:/volume/,volume/,0,0
dbfs:/volumes/,volumes/,0,0


In [ ]:
basePath = 'flights_data'
landingZoneLocation = basePath + "/landingZone"
landingZoneLocation

'flights_data/landingZone'

In [ ]:
dbutils.fs.rm(f"dbfs:/{basePath}/", recurse=True)
try:
  dbutils.fs.ls(basePath)
except:
  dbutils.fs.mkdirs(basePath)
  dbutils.fs.mkdirs(landingZoneLocation)
else:
  raise Exception("The folder " + basePath + " already exists, this notebook will remove it at the end, please change the basePath or remove the folder first")

print("basePath " + basePath + " created")

basePath flights_data created


In [ ]:
spark.sql("SHOW DATABASES")

DataFrame[databaseName: string]

In [ ]:
database = 'flights_db'
createDbSql = f"CREATE DATABASE {database};"
print(createDbSql)

try:
    spark.sql("DROP DATABASE IF EXISTS " + database + " CASCADE")
    spark.sql(createDbSql)
    print("Database " + database + " created")
except Exception as e:
    print(e)


CREATE DATABASE flights_db;
Database flights_db created


In [ ]:
landingZoneLocation = "dbfs:/flights_data/landing-zone"
spark.conf.set("c.ingestDatabase", database)
spark.conf.set("c.ingestLandingZone", landingZoneLocation)
copyIntoTablePath = landingZoneLocation + "/copyIntoTable"
spark.conf.set("c.ingestCopyIntoTablePath", copyIntoTablePath)

# Extract

In [ ]:
access_key = ''
storage_account = 'minhquanstorage03'
container = 'flight'
spark.conf.set("fs.azure.account.key." + storage_account + ".blob.core.windows.net", access_key)

In [ ]:
files = dbutils.fs.ls(f"wasbs://{container}@{storage_account}.blob.core.windows.net/")
parquet_files = [file.path for file in files if 'Flights' in file.path]
print(parquet_files)

data_2018 = spark.read.parquet(f"wasbs://{container}@{storage_account}.blob.core.windows.net/Combined_Flights_2018.parquet")
data_2019 = spark.read.parquet(f"wasbs://{container}@{storage_account}.blob.core.windows.net/Combined_Flights_2019.parquet")
data_2020 = spark.read.parquet(f"wasbs://{container}@{storage_account}.blob.core.windows.net/Combined_Flights_2020.parquet")
data_2021 = spark.read.parquet(f"wasbs://{container}@{storage_account}.blob.core.windows.net/Combined_Flights_2021.parquet")
data_2022 = spark.read.parquet(f"wasbs://{container}@{storage_account}.blob.core.windows.net/Combined_Flights_2022.parquet")


['wasbs://flight@minhquanstorage03.blob.core.windows.net/Combined_Flights_2018.parquet', 'wasbs://flight@minhquanstorage03.blob.core.windows.net/Combined_Flights_2019.parquet', 'wasbs://flight@minhquanstorage03.blob.core.windows.net/Combined_Flights_2020.parquet', 'wasbs://flight@minhquanstorage03.blob.core.windows.net/Combined_Flights_2021.parquet', 'wasbs://flight@minhquanstorage03.blob.core.windows.net/Combined_Flights_2022.parquet', 'wasbs://flight@minhquanstorage03.blob.core.windows.net/c_Combined_Flights_2019.parquet/', 'wasbs://flight@minhquanstorage03.blob.core.windows.net/c_Combined_Flights_2022.parquet/']


In [ ]:
combined_df = data_2018.union(data_2019).union(data_2020).union(data_2021).union(data_2022)
# combined_df.write.parquet(f"wasbs://{container}@{storage_account}.blob.core.windows.net/all_data.parquet")
combined_df.printSchema()

root
 |-- FlightDate: timestamp_ntz (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: long (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: long (nullable = true)
 |-- Quarter: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- DayofMonth: long (nullable = true)
 |-- DayOfWeek: long (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |-- DOT_ID

In [ ]:
combined_df.write.mode("overwrite").saveAsTable("flights_db.flights_data_landing")

# Transform

In [ ]:
from pyspark.sql.functions import col, when, count
data = spark.read.table("flights_db.flights_data_landing")
data.count()

29193782

In [ ]:
data.limit(5).display()

FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,__index_level_0__
2018-01-23T00:00:00,Endeavor Air Inc.,ABY,ATL,false,false,1202,1157.0,0.0,-5.0,1256.0,0.0,38.0,62.0,59.0,145.0,2018,1,1,23,2,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8928A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,14.0,1211.0,1249.0,7.0,1304,-8.0,0.0,-1.0,1300-1359,1,0.0,0
2018-01-24T00:00:00,Endeavor Air Inc.,ABY,ATL,false,false,1202,1157.0,0.0,-5.0,1258.0,0.0,36.0,62.0,61.0,145.0,2018,1,1,24,3,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,13.0,1210.0,1246.0,12.0,1304,-6.0,0.0,-1.0,1300-1359,1,0.0,1
2018-01-25T00:00:00,Endeavor Air Inc.,ABY,ATL,false,false,1202,1153.0,0.0,-9.0,1302.0,0.0,40.0,62.0,69.0,145.0,2018,1,1,25,4,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8836A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,18.0,1211.0,1251.0,11.0,1304,-2.0,0.0,-1.0,1300-1359,1,0.0,2
2018-01-26T00:00:00,Endeavor Air Inc.,ABY,ATL,false,false,1202,1150.0,0.0,-12.0,1253.0,0.0,35.0,62.0,63.0,145.0,2018,1,1,26,5,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1200-1259,17.0,1207.0,1242.0,11.0,1304,-11.0,0.0,-1.0,1300-1359,1,0.0,3
2018-01-27T00:00:00,Endeavor Air Inc.,ABY,ATL,false,false,1400,1355.0,0.0,-5.0,1459.0,0.0,36.0,60.0,64.0,145.0,2018,1,1,27,6,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8839E,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,0.0,-1.0,1400-1459,17.0,1412.0,1448.0,11.0,1500,-1.0,0.0,-1.0,1500-1559,1,0.0,4


In [ ]:
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).display()


FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings,__index_level_0__
0,0,0,0,0,0,0,761652,763084,763084,786177,846183,852561,22,845637,0,0,0,0,0,0,0,0,0,0,0,0,0,0,267613,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,763084,763084,0,780561,780551,793133,793143,0,846183,846183,846183,0,0,90,0


In [ ]:
data = data.dropna()
data.count()

28339510

# Load

In [ ]:
data.write.format('delta').saveAsTable('flights_db.flights_data')

#ML pipeline

In [ ]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [ ]:
data = spark.read.table("flights_db.flights_data")
data.count()

28339510

In [ ]:
dep_cols = [x for x in data.columns if "Dep" in x]
arr_cols = [x for x in data.columns if "Arr" in x]
time_columns = dep_cols + arr_cols + ['AirTime', 'ActualElapsedTime', '__index_level_0__','FlightDate']
data = data.drop(*time_columns)
data.limit(5).display()

Airline,Origin,Dest,Cancelled,Diverted,CRSElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,TaxiOut,WheelsOff,WheelsOn,TaxiIn,DistanceGroup,DivAirportLandings
Endeavor Air Inc.,ABY,ATL,false,false,62.0,145.0,2018,1,1,23,2,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8928A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,14.0,1211.0,1249.0,7.0,1,0.0
Endeavor Air Inc.,ABY,ATL,false,false,62.0,145.0,2018,1,1,24,3,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,13.0,1210.0,1246.0,12.0,1,0.0
Endeavor Air Inc.,ABY,ATL,false,false,62.0,145.0,2018,1,1,25,4,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8836A,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,18.0,1211.0,1251.0,11.0,1,0.0
Endeavor Air Inc.,ABY,ATL,false,false,62.0,145.0,2018,1,1,26,5,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N800AY,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,17.0,1207.0,1242.0,11.0,1,0.0
Endeavor Air Inc.,ABY,ATL,false,false,60.0,145.0,2018,1,1,27,6,DL,DL_CODESHARE,19790,DL,3298,9E,20363,9E,N8839E,3298,10146,1014602,30146,"Albany, GA",GA,13,Georgia,34,10397,1039707,30397,"Atlanta, GA",GA,13,Georgia,34,17.0,1412.0,1448.0,11.0,1,0.0


In [ ]:
str_to_encode = [col for col, dtype in data.dtypes if dtype == "string"]
out_str_to_encode = ["enc_" + col for col in str_to_encode]
num_to_scale = [col for col, dtype in data.dtypes if dtype in ['bigint', 'double']] + out_str_to_encode

In [ ]:
str_ind = StringIndexer(inputCols=str_to_encode, outputCols=out_str_to_encode)

assembler = VectorAssembler(inputCols=num_to_scale, outputCol='c_vec')

scaler = MinMaxScaler(inputCol="c_vec", outputCol="c_scaled")

lr = LogisticRegression(featuresCol='c_scaled', labelCol='Cancelled')
pipeline = Pipeline(stages=[str_ind, assembler, scaler, lr])

In [ ]:
data = data.withColumn("Cancelled", data["Cancelled"].cast("integer"))
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
train_data.limit(5).display()

Airline,Origin,Dest,Cancelled,Diverted,CRSElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,TaxiOut,WheelsOff,WheelsOn,TaxiIn,DistanceGroup,DivAirportLandings
Air Wisconsin Airlines Corp,ABE,ORD,0,false,130.0,654.0,2019,3,7,1,1,UA,UA_CODESHARE,19977,UA,3785,ZW,20046,ZW,N459AW,3785,10135,1013505,30135,"Allentown/Bethlehem/Easton, PA",PA,42,Pennsylvania,23,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,13.0,807.0,841.0,24.0,3,0.0
Air Wisconsin Airlines Corp,ABE,ORD,0,false,130.0,654.0,2019,3,7,2,2,UA,UA_CODESHARE,19977,UA,3785,ZW,20046,ZW,N424AW,3785,10135,1013505,30135,"Allentown/Bethlehem/Easton, PA",PA,42,Pennsylvania,23,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,17.0,808.0,846.0,23.0,3,0.0
Air Wisconsin Airlines Corp,ABE,ORD,0,false,130.0,654.0,2019,3,7,4,4,UA,UA_CODESHARE,19977,UA,3785,ZW,20046,ZW,N452AW,3785,10135,1013505,30135,"Allentown/Bethlehem/Easton, PA",PA,42,Pennsylvania,23,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,15.0,805.0,838.0,23.0,3,0.0
Air Wisconsin Airlines Corp,ABE,ORD,0,false,130.0,654.0,2019,3,7,5,5,UA,UA_CODESHARE,19977,UA,3785,ZW,20046,ZW,N448AW,3785,10135,1013505,30135,"Allentown/Bethlehem/Easton, PA",PA,42,Pennsylvania,23,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,14.0,806.0,840.0,26.0,3,0.0
Air Wisconsin Airlines Corp,ABE,ORD,0,false,130.0,654.0,2019,3,7,6,6,UA,UA_CODESHARE,19977,UA,3785,ZW,20046,ZW,N451AW,3785,10135,1013505,30135,"Allentown/Bethlehem/Easton, PA",PA,42,Pennsylvania,23,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,12.0,802.0,843.0,18.0,3,0.0


In [ ]:
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexer = StringIndexer(
    inputCol="Cancelled",
    outputCol="CancelledIndex"
).setHandleInvalid("keep")

predictions = indexer.fit(predictions).transform(predictions)

evaluator_acc = MulticlassClassificationEvaluator(
    labelCol="CancelledIndex",
    predictionCol="prediction",
    metricName="accuracy"
)
accuracy = evaluator_acc.evaluate(predictions)
print("Accuracy =", accuracy)

evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="CancelledIndex",
    predictionCol="prediction",
    metricName="f1"
)
f1_score = evaluator_f1.evaluate(predictions)
print("F1 Score =", f1_score)